*Pull in the packages we need:*

In [1]:
require(stats)

# Parameters

* **cpd** - Checkins per day. Defaults to 20.

* **days** - Number of days to simulate.  Defaults to 10000, or roughly
    40 working years.

* **winSz** - Size of the commit window as a fraction of `workSec`. Defaults
    to 0.01%, which is roughly 3 seconds for the default 8-hour work day, a
    a long-ish commit time for Fossil.

* **workSec** - Seconds in working day, defaulting to 8 hours.  This value
    only affects the reporting output, not the results of the underlying
    simulation.  It's a scaling parameter to humanize the results.
    
* **spread** - Adjustment factor in computing the standard deviation for our 
    normally-distributed random numbers. The default gives "nice" distributions,
    spread over the working day with a low chance of generating values outside
    the working day.
    
    The smaller this value gets (&lt;4), the more spread out the checkins, and
    so the lower the chance of collisions.  You might want to decrease it a bit
    to simulate early and late workers.
    
    As you increase this value (&gt;4) you're simulating a work environment
    where people tend to check their work in closer and closer to mid-day,
    which increases the chance of collision.

In [2]:
collisions <- function(
        cpd = 20,
        days = 10000,
        winSz = 0.01 / 100,
        workSec = 8 * 60 * 60,
        spread = 4)
{
    cat("Running simulation...\n")

    day = 0
    collisions = 0
    winSec = workSec * winSz
    mean = workSec / 2
    sd = workSec / spread

    while (day < days) {
        # Create the commit time vector as random values in a normal
        # distribution.
        times = sort(rnorm(cpd, mean, sd))

        # Are there any pairs in the time vector that are within the
        # passed window size?  If so, that's a Fossil checkin collision.
        i = 1
        while (i < cpd) {
            if (abs(times[i] - times[i + 1]) < winSec) {
                collisions = collisions + 1
            }
            i = i + 1
        }
        
        day = day + 1
    }
    
    cat("Found", collisions, "collisions in", days, (workSec / 3600),
        "hour working days with", winSec, "second windows.\n")
}

*Run the following cell, possibly changing parameters documented above:*

In [3]:
collisions()

Running simulation...
Found 422 collisions in 10000 8 hour working days with 2.88 second windows.
